In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
model =gp.Model("Modello Ottimo")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-23


In [2]:
files=["./instances2/bmcp_1100_1000_0.3_3000.txt","./instances2/bmcp_1100_1000_0.5_2000.txt",
      "./instances2/bmcp_1400_1500_0.3_3000.txt","./instances2/bmcp_1400_1500_0.5_2000.txt",
       "./instances2/bmcp_1500_1600_0.3_3000.txt","./instances2/bmcp_1500_1600_0.5_2000.txt"]
limits=[33.630189180374146,57.141313791275024,77.35387396812439,126.20470404624939,103.30870199203491,131.63687205314636]

In [3]:
def read_txt2(fileLocation):
    with open(fileLocation,'r') as file:
        dimSubset,dimGroundSet,maxBudget = [int(i) for i in file.readline().split()] 
        # weights
        subsets_weights=np.array( [int(i) for i in file.readline().split()])
        #print(dimSubset,dimGroundSet,maxBudget)
        # profits
        ground_set_profits=np.array( [int(i) for i in file.readline().split()])
        objPerSubset={}
        for sub in range(dimSubset):
            objPerSubset[sub]=[]
        relation_matrix=np.array( [int(i) for i in file.readline().split()])
        for sub in range(dimSubset):
            for obj in range(sub*dimGroundSet,(sub+1)*dimGroundSet):
                if relation_matrix[obj]==1:
                    objPerSubset[sub].append(obj-dimGroundSet*sub)
        return objPerSubset,subsets_weights,ground_set_profits,maxBudget

In [4]:
def solve_with_grb(filelocation,timeLimit):
    objPerSubset,subsets_weights,ground_set_profits,maxBudget=read_txt2(filelocation)
    yvars=model.addVars(range(ground_set_profits.size),name="Y",vtype=GRB.BINARY) # 1 if object included
    xvars=model.addVars(range(subsets_weights.size),name="X",vtype=GRB.BINARY) # 1 if Subset included
    model.setObjective(gp.quicksum(ground_set_profits[i]*yvars[i] for i in range(ground_set_profits.size)),GRB.MAXIMIZE)
    model.addConstr(gp.quicksum(subsets_weights[i]*xvars[i] for i in range(subsets_weights.size))<=maxBudget)
    for obj in range(ground_set_profits.size):
        sub_list=[]
        for subset in range(len(subsets_weights)):
            if obj in objPerSubset[subset]:
                sub_list.append(xvars[subset])
        model.addConstr(gp.quicksum(sub_list[i] for i in range(len(sub_list)) )>=yvars[obj])
    model.setParam("TimeLimit",timeLimit)
    model.optimize()
    value= model.objVal
    status=model.status
    model.reset()
    return value,status
    

In [5]:
solution,state=solve_with_grb("./instances2/bmcp_1100_1000_0.5_2000.txt",30)
print(state)
print(solution)
    

Set parameter TimeLimit to value 30
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1001 rows, 2100 columns and 66223 nonzeros
Model fingerprint: 0x4d0b499c
Variable types: 0 continuous, 2100 integer (2100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 2e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.11s
Presolved: 1001 rows, 2100 columns, 66223 nonzeros
Variable types: 0 continuous, 2100 integer (2100 binary)
Found heuristic solution: objective 89398.000000

Root relaxation: objective 1.469632e+05, 5324 iterations, 1.32 seconds (2.21 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 146963.245    0  490 89398.0000 1469

In [4]:
solution,state=solve_with_grb("./instances2/bmcp_1100_1000_0.3_3000.txt",20)
print(state)
print(solution)

Set parameter TimeLimit to value 20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1001 rows, 2100 columns and 40921 nonzeros
Model fingerprint: 0x28dd46d6
Variable types: 0 continuous, 2100 integer (2100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 3e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.05s
Presolved: 1001 rows, 2100 columns, 40921 nonzeros
Variable types: 0 continuous, 2100 integer (2100 binary)
Found heuristic solution: objective 90151.000000

Root relaxation: objective 1.609233e+05, 8360 iterations, 1.24 seconds (1.55 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 160923.327    0  885 90151.0000 1609

In [6]:
solutions=[]
for i in range(len(files)):
    solution,status=solve_with_grb(files[i],limits[i])
    solutions.append(solution)
print(solutions)

Set parameter TimeLimit to value 3.3630189180374146e+01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2002 rows, 4200 columns and 107144 nonzeros
Model fingerprint: 0x2bef8933
Variable types: 0 continuous, 4200 integer (4200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 3e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 1001 rows and 2100 columns
Presolve time: 0.07s
Presolved: 1001 rows, 2100 columns, 40921 nonzeros
Variable types: 0 continuous, 2100 integer (2100 binary)
Found heuristic solution: objective 90151.000000

Root relaxation: objective 1.609233e+05, 8360 iterations, 1.28 seconds (1.55 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

H    0     0                    178969.00000 242881.374  35.7%     -    2s
H    0     0                    194207.00000 242881.374  25.1%     -    2s
     0     0 242383.350    0 1301 194207.000 242383.350  24.8%     -    3s
     0     0 242383.350    0 1301 194207.000 242383.350  24.8%     -    3s
     0     0 242383.350    0 1303 194207.000 242383.350  24.8%     -    3s
     0     0 242383.350    0 1302 194207.000 242383.350  24.8%     -    3s
     0     0 242383.350    0 1305 194207.000 242383.350  24.8%     -    3s
     0     0 242383.350    0 1302 194207.000 242383.350  24.8%     -    3s
     0     0 242383.350    0 1303 194207.000 242383.350  24.8%     -    3s
     0     0 242383.350    0 1306 194207.000 242383.350  24.8%     -    3s
     0     0 241533.254    0 1308 194207.000 241533.254  24.4%     -    4s
     0     0 241533.254    0 1307 194207.000 241533.254  24.4%     -    4s
     0     0 241533.254    0 1307 194207.000 241533.254  24.4%     -    4s
     0     0 241533.254  

Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 3e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 6005 rows and 12100 columns
Presolve time: 0.19s
Presolved: 1601 rows, 3100 columns, 88226 nonzeros
Variable types: 0 continuous, 3100 integer (3100 binary)
Found heuristic solution: objective 140794.00000

Root relaxation: objective 2.576539e+05, 13882 iterations, 3.25 seconds (4.42 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 257653.946    0 1384 140794.000 257653.946  83.0%     -    3s
H    0     0                    184431.00000 257653.946  39.7%     -    3s
H    0     0                    200411.00000 257653.946  28.6%     -    3s
     0     0 256870.146    0 1389 200411.000 256870.146  28.2%     -    4s
     0     0 256053.970    

H  602   614                    189074.00000 233121.982  23.3%   615   90s
H  603   614                    189154.00000 233121.982  23.2%   615   90s
H  658   660                    190253.00000 233121.982  22.5%   596   94s
   659   682 226666.092  125  863 190253.000 233121.982  22.5%   596   96s
   681   720 226406.393  131  882 190253.000 233121.982  22.5%   610  100s
H  684   720                    190519.00000 233121.982  22.4%   609  100s
H  691   720                    190801.00000 233121.982  22.2%   608  100s
   719   762 225966.727  138  891 190801.000 233121.982  22.2%   599  105s
   807   836 225127.450  156  908 190801.000 233121.982  22.2%   591  110s
   897   993 224522.681  168  911 190801.000 233121.982  22.2%   590  115s
  1058  1163 223492.384  195  914 190801.000 233121.982  22.2%   557  121s
H 1255  1293                    192141.00000 233121.982  21.3%   530  128s
  1292  1379 221957.221  235  938 192141.000 233121.982  21.3%   533  131s

Cutting planes:
  MIR: 1